In [ ]:
# Counting Objects and Estimation of Speed

# Create a GUI and user will be able to select a video.

# GUI should show the first frame of provided video

# User will select ROI to count objects in 

# Develop counting algorithm each vehicle is counted once

# Display the count of all three vehicles on each frame 

# Find total number of cars and the number of counts provided

# Display count of all three types of vehicles

# Estimate the speed of each car and display its speed on its center bounding box

# If speed exceeds 30 km/hour then display a red bounding box around the speeding vehicle (assume 1 square foot is 8by20 pixels)

# Deploy model in a local host and containerize the application using docker container 


In [1]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture('Traffic1.mp4')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file")

# Read the first frame
ret, frame = cap.read()

# Display the first frame
cv2.imshow('Frame', frame)

# Wait for 'Enter' key press
while cv2.waitKey(0) != 13:  # 13 is the ASCII code for 'Enter' key
    pass

# Play the video from the second frame onwards
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Press 'q' on the keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# When everything is done, release the video capture object
cap.release()

# Close all the frames
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture('Traffic1.mp4')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file")

# Read the first frame
ret, frame = cap.read()

# Create a copy of the first frame for drawing the bounding box
frame_with_box = frame.copy()

# Initialize variables for bounding box drawing
drawing = False
bbox = (0, 0, 0, 0)

# Mouse callback function for drawing the bounding box
def draw_bbox(event, x, y, flags, param):
    global frame_with_box, drawing, bbox

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        bbox = (x, y, 0, 0)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bbox = (bbox[0], bbox[1], x - bbox[0], y - bbox[1])
        cv2.rectangle(frame_with_box, (bbox[0], bbox[1]), (x, y), (0, 255, 0), 2)
        cv2.imshow('Frame', frame_with_box)

# Create a window and set the mouse callback
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', draw_bbox)

# Display the initial frame with bounding box
cv2.imshow('Frame', frame_with_box)

# Wait for 'Enter' key press
while cv2.waitKey(0) != 13:  # 13 is the ASCII code for 'Enter' key
    pass

# Play the video from the second frame onwards
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Draw the bounding box on the frame
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Press 'q' on the keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# When everything is done, release the video capture object
cap.release()

# Close all the frames
cv2.destroyAllWindows()

: 

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO('yolov5s.yaml')

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov5s.pt')

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture('Traffic1.mp4')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file")

# Read the first frame
ret, frame = cap.read()

# Create a copy of the first frame for drawing the bounding box
frame_with_box = frame.copy()

# Initialize variables for bounding box drawing
drawing = False
bbox1 = (0, 0, 0, 0)

# Mouse callback function for drawing the bounding box
def draw_bbox(event, x, y, flags, param):
    global frame_with_box, drawing, bbox1

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        bbox1 = (x, y, 0, 0)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bbox1 = (bbox1[0], bbox1[1], x - bbox1[0], y - bbox1[1])
        cv2.rectangle(frame_with_box, (bbox1[0], bbox1[1]), (x, y), (0, 255, 0), 2)
        cv2.imshow('Frame', frame_with_box)

# Create a window and set the mouse callback
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', draw_bbox)

# Display the initial frame with bounding box
cv2.imshow('Frame', frame_with_box)

# Wait for 'Enter' key press
while cv2.waitKey(0) != 13:  # 13 is the ASCII code for 'Enter' key
    pass

frame_count = 0
# Play the video from the second frame onwards
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_count += 1
    print("********Frame Count********: ",frame_count)
    if ret:
        print("*****THE***** bbox length: ",len(bbox1))
        print("*****THE***** bbox contents: ",bbox1)

        # Draw the bounding box on the frame
        #print(bbox)
        print("BBOX_0", bbox1[0])
        print("BBOX_1", bbox1[1])
        print("BBOX_2", bbox1[2])
        print("BBOX_3", bbox1[3])
        cv2.rectangle(frame, (bbox1[0], bbox1[1]), (bbox1[0] - bbox1[2], bbox1[1] - bbox1[3]), (0, 255, 0), 2)
        
        #cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)

        # Extract the region of interest (ROI) within the bounding box
        roi = frame[bbox1[1]:bbox1[1] + bbox1[3], bbox1[0]:bbox1[0] + bbox1[2]]

        # Perform object detection on the ROI using YOLO
        if roi.size > 0:  # Check if the ROI is non-empty
            results = model(roi)
            # print(type(results))
            # print(type(results[0]))
            # print(dir(results))
            # print([attr for attr in dir(results) if not attr.startswith('__') and attr != '__class__'])

            bboxes = []
            labels = []
                    
            for result in results:
                bboxes = result.boxes.numpy()
                labels = result.names

                for bbox, name in zip(bboxes, labels):
                    #print("Length of bbox: ",len(bbox))
                    #print("Bbox contents: ",bbox.xyxy)
                    #print(type(bbox.xyxy))
                    if bbox.xyxy.size >= 4:
                        x1, y1, x2, y2 = bbox.xyxy[0]
                        #print(f"Bounding box: ({x1}, {y1}) - ({x2}, {y2}), Name: {name}")
                    else:
                        print("Invalid bounding box format1")

            num_objects = len(bboxes)
            # Draw bounding boxes and labels on the frame if objects are detected
            if num_objects > 0:
                for bbox in bboxes:
                    if bbox.xyxy.size >= 4:
                        x1, y1, x2, y2 = bbox.xyxy[0]
                        x1, y1, x2, y2 = int(x1.item()), int(y1.item()), int(x2.item()), int(y2.item())
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        roi = frame[y1:y2, x1:x2]
                        # Process the ROI as needed
                    
                        #print(f"Bounding box: ({x1}, {y1}) - ({x2}, {y2})")
                    else:
                        print("Invalid bounding box format2")
            
        # Display the frame with bounding boxes and labels
        cv2.imshow('Frame', frame)
    
        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the VideoCapture object and close the windows
cap.release()
cv2.destroyAllWindows()


                   from  n    params  module                                       arguments                     


  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1182720  ultralytics.nn.modules.block.C3              [512,

********Frame Count********:  1
*****THE***** bbox length:  4
*****THE***** bbox contents:  (323, 206, 602, 243)
BBOX_0 323
BBOX_1 206
BBOX_2 602
BBOX_3 243



0: 288x640 7 cars, 225.6ms
Speed: 2.2ms preprocess, 225.6ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 640)



********Frame Count********:  2
*****THE***** bbox length:  4
*****THE***** bbox contents:  (323, 206, 602, 243)
BBOX_0 323
BBOX_1 206
BBOX_2 602
BBOX_3 243


...

********Frame Count********:  74
*****THE***** bbox length:  4
*****THE***** bbox contents:  (323, 206, 602, 243)
BBOX_0 323
BBOX_1 206
BBOX_2 602
BBOX_3 243


0: 288x640 7 cars, 1 truck, 211.3ms
Speed: 1.9ms preprocess, 211.3ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)



********Frame Count********:  75
*****THE***** bbox length:  4
*****THE***** bbox contents:  (323, 206, 602, 243)
BBOX_0 323
BBOX_1 206
BBOX_2 602
BBOX_3 243


0: 288x640 6 cars, 1 truck, 216.4ms
Speed: 2.4ms preprocess, 216.4ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)


: 

None
